In [ ]:
# Installations
!pip install geopandas
!pip install pygeos

# Dependencies
import geopandas as gpd
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import altair as alt
import pygeos


In [2]:
data_path = 'data'

Mounted at /drive


## Data sources


In [3]:
ritracts = gpd.read_file(f'{data_path}/raw/ri_tract_map')

riplaces = gpd.read_file(f'{data_path}/raw/ri_places')


In [4]:
ritracts.columns

Index(['STATEFP', 'COUNTYFP', 'TRACTCE', 'AFFGEOID', 'GEOID', 'NAME', 'LSAD',
       'ALAND', 'AWATER', 'geometry'],
      dtype='object')

In [5]:
riplaces = riplaces.drop(columns=['STATEFP',"GEOID", 'PLACENS','AFFGEOID', 'ALAND', 'AWATER','LSAD'])


riplaces.rename(columns = {'PLACEFP':'place_fp', 'NAME':'place_name'}, inplace = True)

ritracts = ritracts.drop(columns=['STATEFP',"GEOID", 'AFFGEOID', 'ALAND', 'AWATER','LSAD'])


ritracts.rename(columns = {'TRACTCE':'tract_ce', 'NAME':'tract_name'}, inplace = True)


#### Maps of RI Counties, Tracts, and Places

In [8]:
alt.Chart(ritracts).mark_geoshape( stroke='grey').encode(color='COUNTYFP',
tooltip='tract_name').properties(title='RI Counties and tracts')



alt.Chart(...)

In [ ]:
ritracts.columns

Index(['COUNTYFP', 'tract_ce', 'tract_name', 'geometry'], dtype='object')

In [ ]:
# this spatially joins tracts and places by intersection. In other words, if a tract and place intersect, that tract will be labeled as a part of the place. 

ri_tracts_more = gpd.sjoin(ritracts, riplaces, how="left", predicate='within')


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: UserWarning: CRS mismatch between the CRS of left geometries and the CRS of right geometries.
Use `to_crs()` to reproject one of the input geometries to match the CRS of the other.

Left CRS: EPSG:4269
Right CRS: None

  This is separate from the ipykernel package so we can avoid doing imports until


In [ ]:
alt.Chart(ri_tracts_more).mark_geoshape( stroke='grey').encode(color='place_name',
tooltip='place_name').properties(title='RI Tracts and Places')

alt.Chart(...)

In [ ]:
#Create a column of the df that has core cities and not core city as a label for each tract

ri_tracts_more.head()

def core_cities_assign(row):
  if row =="Pawtucket":
    result = 'Pawtucket'
  elif row == "Providence":
    result ="Providence"
  elif row == "Woonsocket":
    result = "Woonsocket"
  elif row =="Central Falls":
    result = "Central Falls"
  else:
    result = "Not_core_city"


  return result


 


ri_tracts_more['core_cities'] = ri_tracts_more['place_name'].apply(core_cities_assign)



ri_tracts_more[ri_tracts_more['core_cities']=='Central Falls']

COUNTYFP tract_ce tract_name  \
97       007   011000        110   
151      007   010800        108   
171      007   010900        109   

                                              geometry  index_right place_fp  \
97   POLYGON ((-71.40457 41.89482, -71.39724 41.896...          0.0    14140   
151  POLYGON ((-71.39598 41.88574, -71.39380 41.887...          0.0    14140   
171  POLYGON ((-71.39724 41.89678, -71.39430 41.897...          0.0    14140   

        place_name    core_cities  
97   Central Falls  Central Falls  
151  Central Falls  Central Falls  
171  Central Falls  Central Falls

In [ ]:
#This graphs the core cities in an RI map. We'll want to iron out color choices if we decide to use this. 

alt.Chart(ri_tracts_more).mark_geoshape(stroke='grey').\
    encode(color=alt.Color('core_cities',\
          scale=alt.Scale(domain=['Providence', 'Pawtucket','Woonsocket','Central Falls',"Not_core_city"],\
          range=['blue','yellow','green','red','#D3D3D3'])\
          ,legend=alt.Legend(title="Core Cities")),\
           tooltip='place_name')\
           .properties(title='Place to tract for core cities')

alt.Chart(...)

In [ ]:
#here are the number of tracts for each county providence is 007
ritracts.groupby('COUNTYFP').count()['tract_ce']

#here are the number of tracts for each core city

print(len(ritracts))
ri_tracts_more.groupby('core_cities').count()['tract_ce']





243


core_cities
Central Falls      3
Not_core_city    173
Pawtucket         21
Providence        37
Woonsocket         9
Name: tract_ce, dtype: int64

In [ ]:

ri_tracts_more.to_csv(f'{data_path}/interim/tract_place_mapper.csv')

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=96aa4e71-9142-4f1e-8c2a-54e13d3b1860' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>